# Venture Funding with Deep Learning

You work as a risk management associate at Alphabet Soup, a venture capital firm. Alphabet Soup’s business team receives many funding applications from startups every day. This team has asked you to help them create a model that predicts whether applicants will be successful if funded by Alphabet Soup.

The business team has given you a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. With your knowledge of machine learning and neural networks, you decide to use the features in the provided dataset to create a binary classifier model that will predict whether an applicant will become a successful business. The CSV file contains a variety of information about these businesses, including whether or not they ultimately became successful.

## Instructions:

The steps for this challenge are broken out into the following sections:
* Prepare the data for use on a neural network model.
* Compile and evaluate a binary classification model using a neural network.
* Optimize the neural network model.

### Prepare the Data for Use on a Neural Network Model 

Using your knowledge of Pandas and scikit-learn’s `StandardScaler()`, preprocess the dataset so that you can use it to compile and evaluate the neural network model later.

Open the starter code file, and complete the following data preparation steps:

1. Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.   

2. Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.
 
3. Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

4. Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

5. Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 

6. Split the features and target sets into training and testing datasets.

7. Use scikit-learn's `StandardScaler` to scale the features data.

### Compile and Evaluate a Binary Classification Model Using a Neural Network

Use your knowledge of TensorFlow to design a binary classification deep neural network model. This model should use the dataset’s features to predict whether an Alphabet Soup&ndash;funded startup will be successful based on the features in the dataset. Consider the number of inputs before determining the number of layers that your model will contain or the number of neurons on each layer. Then, compile and fit your model. Finally, evaluate your binary classification model to calculate the model’s loss and accuracy. 
 
To do so, complete the following steps:

1. Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.

2. Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.

> **Hint** When fitting the model, start with a small number of epochs, such as 20, 50, or 100.

3. Evaluate the model using the test data to determine the model’s loss and accuracy.

4. Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 

### Optimize the Neural Network Model

Using your knowledge of TensorFlow and Keras, optimize your model to improve the model's accuracy. Even if you do not successfully achieve a better accuracy, you'll need to demonstrate at least two attempts to optimize the model. You can include these attempts in your existing notebook. Or, you can make copies of the starter notebook in the same folder, rename them, and code each model optimization in a new notebook. 

> **Note** You will not lose points if your model does not achieve a high accuracy, as long as you make at least two attempts to optimize the model.

To do so, complete the following steps:

1. Define at least three new deep neural network models (the original plus 2 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
> * Add more neurons (nodes) to a hidden layer.
> * Add more hidden layers.
> * Use different activation functions for the hidden layers.
> * Add to or reduce the number of epochs in the training regimen.

2. After finishing your models, display the accuracy scores achieved by each model, and compare the results.

3. Save each of your models as an HDF5 file.

In [121]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

pd.set_option('display.max_columns', None)

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `applicants_data.csv` file into a Pandas DataFrame. Review the DataFrame, looking for categorical variables that will need to be encoded, as well as columns that could eventually define your features and target variables.  


In [122]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv(Path('Resources/applicants_data.csv'))

# Review the DataFrame
print(applicant_data_df.shape)
applicant_data_df.tail(100)

(34299, 12)


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
34199,956208158,AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34200,956208409,PTA CALIFORNIA CONGRESS OF PARENTS TEACHERS & ...,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34201,956226785,AMERICAN ASSOCIATION OF UNIVERSITY WOMEN INC,T4,CompanySponsored,C3000,ProductDev,Association,1,1-9999,N,7003,0
34202,956226870,AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34203,956256338,PTA CALIFORNIA CONGRESS OF PARENTS TEACHERS & ...,T3,CompanySponsored,C2000,Preservation,Association,1,10000-24999,N,8365,0
...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,996012607,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,996015768,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [123]:
# Review the data types associated with the columns
applicant_data_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [124]:
# Checking the object type data
applicant_data_df.describe(include='object')

# 'SPECIAL_CONSIDERATIONS' only has two unique values, so might want to think 
#  about dropping an encoded value later, to avoid the 'Dummy Variable Trap'.

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS
count,34299,34299,34299,34299,34299,34299,34299,34299
unique,19568,17,6,71,5,4,9,2
top,PARENT BOOSTER USA INC,T3,Independent,C1000,Preservation,Trust,0,N
freq,1260,27037,18480,17326,28095,23515,24388,34272


In [125]:
# Checking the numbrer type data
applicant_data_df.describe(include='int64')

# hmm, 'STATUS' and 'IS_SUCCESSFUL' look like they should actually be categorical variables, 
#  since the only values are 0 and 1.
# Ah, encoded categorical variables are cast to 0 and 1 int64 anyway, so it's fine that these 
#  two variables are int64s. 

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL
count,3.429900e+04,34299.000000,3.429900e+04,34299.000000
mean,5.191852e+08,0.999854,2.769199e+06,0.532406
std,2.451472e+08,0.012073,8.713045e+07,0.498956
min,1.052060e+07,0.000000,5.000000e+03,0.000000
25%,2.748482e+08,1.000000,5.000000e+03,0.000000
50%,4.656317e+08,1.000000,5.000000e+03,1.000000
75%,7.526117e+08,1.000000,7.742000e+03,1.000000
max,9.960869e+08,1.000000,8.597806e+09,1.000000


### Step 2: Drop the “EIN” (Employer Identification Number) and “NAME” columns from the DataFrame, because they are not relevant to the binary classification model.

In [126]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=['EIN', 'NAME'])

# Review the DataFrame
print(applicant_data_df.shape)
applicant_data_df.head()

(34299, 10)


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


### Step 3: Encode the dataset’s categorical variables using `OneHotEncoder`, and then place the encoded variables into a new DataFrame.

In [127]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == 'object'].index)

# Display the categorical variables list
categorical_variables

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [128]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

In [129]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])

In [130]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(encoded_data, columns=enc.get_feature_names(categorical_variables))

# Review the DataFrame
print(encoded_df.info(verbose=True))
encoded_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 114 columns):
 #    Column                        Dtype  
---   ------                        -----  
 0    APPLICATION_TYPE_T10          float64
 1    APPLICATION_TYPE_T12          float64
 2    APPLICATION_TYPE_T13          float64
 3    APPLICATION_TYPE_T14          float64
 4    APPLICATION_TYPE_T15          float64
 5    APPLICATION_TYPE_T17          float64
 6    APPLICATION_TYPE_T19          float64
 7    APPLICATION_TYPE_T2           float64
 8    APPLICATION_TYPE_T25          float64
 9    APPLICATION_TYPE_T29          float64
 10   APPLICATION_TYPE_T3           float64
 11   APPLICATION_TYPE_T4           float64
 12   APPLICATION_TYPE_T5           float64
 13   APPLICATION_TYPE_T6           float64
 14   APPLICATION_TYPE_T7           float64
 15   APPLICATION_TYPE_T8           float64
 16   APPLICATION_TYPE_T9           float64
 17   AFFILIATION_CompanySponsored  float64
 18   AFFI

,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C0,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1230,CLASSIFICATION_C1234,CLASSIFICATION_C1235,CLASSIFICATION_C1236,CLASSIFICATION_C1237,CLASSIFICATION_C1238,CLASSIFICATION_C1240,CLASSIFICATION_C1245,CLASSIFICATION_C1246,CLASSIFICATION_C1248,CLASSIFICATION_C1250,CLASSIFICATION_C1256,CLASSIFICATION_C1257,CLASSIFICATION_C1260,CLASSIFICATION_C1267,CLASSIFICATION_C1270,CLASSIFICATION_C1278,CLASSIFICATION_C1280,CLASSIFICATION_C1283,CLASSIFICATION_C1300,CLASSIFICATION_C1370,CLASSIFICATION_C1400,CLASSIFICATION_C1500,CLASSIFICATION_C1570,CLASSIFICATION_C1580,CLASSIFICATION_C1600,CLASSIFICATION_C1700,CLASSIFICATION_C1720,CLASSIFICATION_C1728,CLASSIFICATION_C1732,CLASSIFICATION_C1800,CLASSIFICATION_C1820,CLASSIFICATION_C1900,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2150,CLASSIFICATION_C2170,CLASSIFICATION_C2190,CLASSIFICATION_C2300,CLASSIFICATION_C2380,CLASSIFICATION_C2400,CLASSIFICATION_C2500,CLASSIFICATION_C2561,CLASSIFICATION_C2570,CLASSIFICATION_C2600,CLASSIFICATION_C2700,CLASSIFICATION_C2710,CLASSIFICATION_C2800,CLASSIFICATION_C3000,CLASSIFICATION_C3200,CLASSIFICATION_C3700,CLASSIFICATION_C4000,CLASSIFICATION_C4100,CLASSIFICATION_C4120,CLASSIFICATION_C4200,CLASSIFICATION_C4500,CLASSIFICATION_C5000,CLASSIFICATION_C5200,CLASSIFICATION_C6000,CLASSIFICATION_C6100,CLASSIFICATION_C7000,CLASSIFICATION_C7100,CLASSIFICATION_C7120,CLASSIFICATION_C7200,CLASSIFICATION_C7210,CLASSIFICATION_C8000,CLASSIFICATION_C8200,CLASSIFICATION_C8210,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### Step 4: Add the original DataFrame’s numerical variables to the DataFrame containing the encoded variables.

> **Note** To complete this step, you will employ the Pandas `concat()` function that was introduced earlier in this course. 

In [131]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables_df = applicant_data_df.select_dtypes(include='int64')
processed_df = pd.concat([numerical_variables_df, encoded_df], axis=1)

# Review the Dataframe
print(f"Shape: {processed_df.shape}\n")
print(f"dtypes:\n{processed_df.dtypes}")

Shape: (34299, 117)

dtypes:
STATUS                        int64
ASK_AMT                       int64
IS_SUCCESSFUL                 int64
APPLICATION_TYPE_T10        float64
APPLICATION_TYPE_T12        float64
                             ...   
INCOME_AMT_25000-99999      float64
INCOME_AMT_50M+             float64
INCOME_AMT_5M-10M           float64
SPECIAL_CONSIDERATIONS_N    float64
SPECIAL_CONSIDERATIONS_Y    float64
Length: 117, dtype: object


### Step 5: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “IS_SUCCESSFUL”. The remaining columns should define the features dataset. 



In [132]:
# Define the target set y using the IS_SUCCESSFUL column
y = processed_df['IS_SUCCESSFUL']

# Display a sample of y
y[:5]

0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [133]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = processed_df.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
print(f"Shape: {X.shape}\n")

Shape: (34299, 116)



### Step 6: Split the features and target sets into training and testing datasets.


In [146]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 7: Use scikit-learn's `StandardScaler` to scale the features data.

In [147]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [148]:
# Define the the number of inputs (features) to the model
number_input_features = X.shape[1]

# Review the number of features
number_input_features

22

In [149]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [150]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + 1) // 2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1

11

In [151]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = (hidden_nodes_layer1 + 1) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

6

In [152]:
# Create the Sequential model instance
nn = Sequential()

In [153]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

In [154]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation='relu'))

In [155]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation='sigmoid'))

In [156]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 11)                253       
                                                                 
 dense_22 (Dense)            (None, 6)                 72        
                                                                 
 dense_23 (Dense)            (None, 1)                 7         
                                                                 
Total params: 332
Trainable params: 332
Non-trainable params: 0
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [157]:
# Compile the Sequential model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [158]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5971 - accuracy: 0.6854
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5656 - accuracy: 0.7227
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5622 - accuracy: 0.7247
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5614 - accuracy: 0.7220
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5604 - accuracy: 0.7250
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5601 - accuracy: 0.7247
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5594 - accuracy: 0.7249
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5587 - accuracy: 0.7256
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5583 - accuracy: 0.7257
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5583 - accuracy: 0.7256

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [53]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5540 - accuracy: 0.7286 - 432ms/epoch - 2ms/step
Loss: 0.5540483593940735, Accuracy: 0.7286297082901001


### Step 4: Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`. 


In [ ]:
# Set the model's file path
file_path = # YOUR CODE HERE

# Export your model to a HDF5 file
# YOUR CODE HERE


---

## Optimize the neural network model


### Step 1: Define at least three new deep neural network models (resulting in the original plus 3 optimization attempts). With each, try to improve on your first model’s predictive accuracy.

> **Rewind** Recall that perfect accuracy has a value of 1, so accuracy improves as its value moves closer to 1. To optimize your model for a predictive accuracy as close to 1 as possible, you can use any or all of the following techniques:
> * Adjust the input data by dropping different features columns to ensure that no variables or outliers confuse the model.
> * Add more neurons (nodes) to a hidden layer.
> * Add more hidden layers.
> * Use different activation functions for the hidden layers.
> * Add to or reduce the number of epochs in the training regimen.

### Alternative Model 1

In [68]:
# drop extra features, so as not to fall into the Dummy Variable Trap
# Create a OneHotEncoder instance
enc_drop_first = OneHotEncoder(sparse=False, drop='first')

# Encode the categorcal variables using OneHotEncoder
encoded_data_drop_first = enc_drop_first.fit_transform(applicant_data_df[categorical_variables])

# Create a DataFrame with the encoded variables
encoded_df_drop_first = pd.DataFrame(encoded_data_drop_first, columns=enc_drop_first.get_feature_names(categorical_variables))

# Review the DataFrame
print(encoded_df_drop_first.info(verbose=False))
encoded_df_drop_first.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Columns: 107 entries, APPLICATION_TYPE_T12 to SPECIAL_CONSIDERATIONS_Y
dtypes: float64(107)
memory usage: 28.0 MB
None


,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,APPLICATION_TYPE_T9,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1230,CLASSIFICATION_C1234,CLASSIFICATION_C1235,CLASSIFICATION_C1236,CLASSIFICATION_C1237,CLASSIFICATION_C1238,CLASSIFICATION_C1240,CLASSIFICATION_C1245,CLASSIFICATION_C1246,CLASSIFICATION_C1248,CLASSIFICATION_C1250,CLASSIFICATION_C1256,CLASSIFICATION_C1257,CLASSIFICATION_C1260,CLASSIFICATION_C1267,CLASSIFICATION_C1270,CLASSIFICATION_C1278,CLASSIFICATION_C1280,CLASSIFICATION_C1283,CLASSIFICATION_C1300,CLASSIFICATION_C1370,CLASSIFICATION_C1400,CLASSIFICATION_C1500,CLASSIFICATION_C1570,CLASSIFICATION_C1580,CLASSIFICATION_C1600,CLASSIFICATION_C1700,CLASSIFICATION_C1720,CLASSIFICATION_C1728,CLASSIFICATION_C1732,CLASSIFICATION_C1800,CLASSIFICATION_C1820,CLASSIFICATION_C1900,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C2150,CLASSIFICATION_C2170,CLASSIFICATION_C2190,CLASSIFICATION_C2300,CLASSIFICATION_C2380,CLASSIFICATION_C2400,CLASSIFICATION_C2500,CLASSIFICATION_C2561,CLASSIFICATION_C2570,CLASSIFICATION_C2600,CLASSIFICATION_C2700,CLASSIFICATION_C2710,CLASSIFICATION_C2800,CLASSIFICATION_C3000,CLASSIFICATION_C3200,CLASSIFICATION_C3700,CLASSIFICATION_C4000,CLASSIFICATION_C4100,CLASSIFICATION_C4120,CLASSIFICATION_C4200,CLASSIFICATION_C4500,CLASSIFICATION_C5000,CLASSIFICATION_C5200,CLASSIFICATION_C6000,CLASSIFICATION_C6100,CLASSIFICATION_C7000,CLASSIFICATION_C7100,CLASSIFICATION_C7120,CLASSIFICATION_C7200,CLASSIFICATION_C7210,CLASSIFICATION_C8000,CLASSIFICATION_C8200,CLASSIFICATION_C8210,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0

In [62]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
numerical_variables_df = applicant_data_df.select_dtypes(include='int64')
processed_df_drop_first = pd.concat([numerical_variables_df, encoded_df_drop_first], axis=1)

# Review the Dataframe
print(f"Shape: {processed_df_drop_first.shape}\n")
print(f"dtypes:\n{processed_df_drop_first.dtypes}")

Shape: (34299, 110)

dtypes:
STATUS                        int64
ASK_AMT                       int64
IS_SUCCESSFUL                 int64
APPLICATION_TYPE_T12        float64
APPLICATION_TYPE_T13        float64
                             ...   
INCOME_AMT_1M-5M            float64
INCOME_AMT_25000-99999      float64
INCOME_AMT_50M+             float64
INCOME_AMT_5M-10M           float64
SPECIAL_CONSIDERATIONS_Y    float64
Length: 110, dtype: object


In [162]:
# Define the target set y using the IS_SUCCESSFUL column
y = processed_df_drop_first['IS_SUCCESSFUL']

# Define features set X by selecting all columns but IS_SUCCESSFUL
X = processed_df_drop_first.drop(columns=['IS_SUCCESSFUL'])

# Review the features DataFrame
print(f"Shape: {X.shape}\n")

# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
# X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)
X_train_scaled = X_train
X_test_scaled = X_test

Shape: (34299, 109)



In [163]:
# Define the the number of inputs (features) to the model
number_input_features_A1 = len(X_train.iloc[0])

# Review the number of features
number_input_features_A1

109

In [164]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + 1) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

11

In [165]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 = (hidden_nodes_layer1_A1 + 1) // 2
hidden_nodes_layer2_A1

6

In [166]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [167]:
# First hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer1_A1, activation='relu', input_dim=number_input_features_A1))

# Add the second hidden layer
nn_A1.add(Dense(units=hidden_nodes_layer2_A1, activation='relu'))

# Output layer
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units=number_output_neurons_A1, activation='sigmoid'))

# Check the structure of the model
nn_A1.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 11)                1210      
                                                                 
 dense_25 (Dense)            (None, 6)                 72        
                                                                 
 dense_26 (Dense)            (None, 1)                 7         
                                                                 
Total params: 1,289
Trainable params: 1,289
Non-trainable params: 0
_________________________________________________________________


In [168]:
# Compile the Sequential model
nn_A1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [169]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 25132.8848 - accuracy: 0.5037
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 23692.4590 - accuracy: 0.4842
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 7493.8828 - accuracy: 0.4990
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 8859.6973 - accuracy: 0.5224
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 2467.5813 - accuracy: 0.5046
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 10991.3330 - accuracy: 0.5057
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 9604.2852 - accuracy: 0.5342
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 2695.6799 - accuracy: 0.5368A: 0s - loss: 2297.0134 - accura
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 15569.8438 - accuracy: 0.5119
Epoch 10/50
804/804 [================

#### Alternative Model 2

In [91]:
# Define the the number of inputs (features) to the model
number_input_features_A2 = len(X_train.iloc[0])

# Review the number of features
number_input_features_A2

109

In [106]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features_A2 + 1) // 2
hidden_nodes_layer2_A2 = (hidden_nodes_layer1_A2 + 1) // 4
hidden_nodes_layer3_A2 = (hidden_nodes_layer2_A2 + 1) // 4
hidden_nodes_layer3_A2

3

In [107]:
# Create the Sequential model instance
nn_A2 = Sequential()

In [108]:
# First hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer1_A2, activation='relu', input_dim=number_input_features_A2))

# Add the second hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer2_A2, activation='relu'))

# Add the third hidden layer
nn_A2.add(Dense(units=hidden_nodes_layer3_A2, activation='relu'))

# Output layer
# Add the output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units=number_output_neurons_A2, activation='sigmoid'))

# Check the structure of the model
nn_A2.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 55)                6050      
                                                                 
 dense_18 (Dense)            (None, 14)                784       
                                                                 
 dense_19 (Dense)            (None, 3)                 45        
                                                                 
 dense_20 (Dense)            (None, 1)                 4         
                                                                 
Total params: 6,883
Trainable params: 6,883
Non-trainable params: 0
_________________________________________________________________


In [109]:
# Compile the model
nn_A2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [110]:
# Fit the model
fit_model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5904 - accuracy: 0.6950
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5595 - accuracy: 0.7208
Epoch 3/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5548 - accuracy: 0.7221
Epoch 4/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5521 - accuracy: 0.7225
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5519 - accuracy: 0.7224
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5512 - accuracy: 0.7229
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7235
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5494 - accuracy: 0.7245
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5481 - accuracy: 0.7259
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5477 - accuracy: 0.7272

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [ ]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy =# YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = # YOUR CODE HERE

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Step 3: Save each of your alternative models as an HDF5 file.


In [ ]:
# Set the file path for the first alternative model
file_path = # YOUR CODE HERE

# Export your model to a HDF5 file
# YOUR CODE HERE


In [ ]:
# Set the file path for the second alternative model
file_path = # YOUR CODE HERE

# Export your model to a HDF5 file
# YOUR CODE HERE


In [161]:
pd.DataFrame(X_train_scaled)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,-0.029571,0.520286,-0.218484,-0.193996,-0.189309,-0.178463,-0.150399,-0.14518,-0.123588,-1.073189,1.080344,-1.005066,2.462741,-0.466155,-0.241175,-0.243423,-0.154141,0.472261,-0.447443,1.527695,-0.119471,-1.473467
1,-0.029571,-1.922020,-0.218484,5.154733,-0.189309,-0.178463,-0.150399,-0.14518,-0.123588,0.931802,-0.925631,-1.005066,-0.406052,2.145208,-0.241175,-0.243423,-0.154141,-2.117473,2.234921,-0.654581,-0.119471,0.678672
2,-0.029571,0.520286,-0.218484,-0.193996,-0.189309,-0.178463,-0.150399,-0.14518,-0.123588,0.931802,-0.925631,0.994959,-0.406052,-0.466155,-0.241175,-0.243423,-0.154141,0.472261,-0.447443,-0.654581,-0.119471,0.678672
3,-0.029571,0.520286,-0.218484,-0.193996,-0.189309,-0.178463,-0.150399,-0.14518,-0.123588,0.931802,-0.925631,-1.005066,-0.406052,2.145208,-0.241175,-0.243423,-0.154141,0.472261,-0.447443,-0.654581,-0.119471,0.678672
4,-0.029571,-1.922020,-0.218484,-0.193996,-0.189309,-0.178463,-0.150399,-0.14518,8.091371,-1.073189,1.080344,0.994959,-0.406052,-0.466155,-0.241175,-0.243423,-0.154141,0.472261,-0.447443,1.527695,-0.119471,-1.473467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25719,-0.029571,0.520286,-0.218484,-0.193996,-0.189309,-0.178463,-0.150399,-0.14518,-0.123588,-1.073189,1.080344,-1.005066,-0.406052,-0.466155,4.146367,-0.243423,-0.154141,0.472261,-0.447443,-0.654581,-0.119471,0.678672
25720,-0.029571,-1.922020,-0.218484,-0.193996,5.282364,-0.178463,-0.150399,-0.14518,-0.123588,0.931802,-0.925631,-1.005066,-0.406052,-0.466155,-0.241175,4.108078,-0.154141,-2.117473,2.234921,1.527695,-0.119471,-1.473467
25721,-0.029571,0.520286,-0.218484,-0.193996,-0.189309,-0.178463,-0.150399,-0.14518,-0.123588,-1.073189,1.080344,-1.005066,-0.406052,2.145208,-0.241175,-0.243423,-0.154141,0.472261,-0.447443,1.527695,-0.119471,-1.473467
25722,-0.029571,0.520286,-0.218484,-0.193996,-0.189309,-0.178463,-0.150399,-0.14518,-0.123588,-1.073189,1.080344,-1.005066,-0.406052,2.145208,-0.241175,-0.243423,-0.154141,0.472261,-0.447443,1.527695,-0.119471,-1.473467


In [141]:
X

,ASK_AMT,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T6,APPLICATION_TYPE_T5,APPLICATION_TYPE_T19,APPLICATION_TYPE_T8,APPLICATION_TYPE_T7,APPLICATION_TYPE_T10,AFFILIATION_Independent,AFFILIATION_CompanySponsored,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Trust
0,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,108590,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,5000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
3,6692,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,142590,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
34295,5000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
34296,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
34297,5000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0


In [140]:
X = X[['ASK_AMT', 
'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4',
'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T19',
'APPLICATION_TYPE_T8', 'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T10',
'AFFILIATION_Independent', 'AFFILIATION_CompanySponsored', 
'CLASSIFICATION_C1000',
'CLASSIFICATION_C1200',
'CLASSIFICATION_C2000',
'CLASSIFICATION_C2100',
'CLASSIFICATION_C3000',
'CLASSIFICATION_C7000',
'USE_CASE_Preservation', 'USE_CASE_ProductDev',
 'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
'ORGANIZATION_Trust']]

In [144]:
def Ask_Group (x):
    if x == 5000:
        x = 0
    
X['ASK_AMT'].apply(Ask_Group)

0        None
1        None
2        None
3        None
4        None
         ... 
34294    None
34295    None
34296    None
34297    None
34298    None
Name: ASK_AMT, Length: 34299, dtype: object

In [160]:
X_train_scaled

array([[-0.02957055,  0.52028597, -0.21848437, ...,  1.52769491,
        -0.11947101, -1.47346684],
       [-0.02957055, -1.92201992, -0.21848437, ..., -0.65458096,
        -0.11947101,  0.67867153],
       [-0.02957055,  0.52028597, -0.21848437, ..., -0.65458096,
        -0.11947101,  0.67867153],
       ...,
       [-0.02957055,  0.52028597, -0.21848437, ...,  1.52769491,
        -0.11947101, -1.47346684],
       [-0.02957055,  0.52028597, -0.21848437, ...,  1.52769491,
        -0.11947101, -1.47346684],
       [-0.02957055, -1.92201992,  4.5769865 , ...,  1.52769491,
        -0.11947101, -1.47346684]])